## The Dataset

The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification). There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas.

## Load ImageNet Base Model

In [1]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

58892288/58889256 [==============================] - 0s 0us/step


## Freeze Base Model

In [2]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

In [3]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x =  keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(1, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________


## Compile Model

In [5]:
model.compile(loss = 'categorical_crossentropy' , metrics = ['accuracy'])

## Augment the Data

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images horizontally
    vertical_flip=False, # Don't randomly flip images vertically
)

## Load Dataset

In [8]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('data/fruits/train', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('data/fruits/valid', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


## Train the Model

In [9]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=20)

Epoch 1/20
37/36 [==============================] - 29s 791ms/step - loss: 0.5626 - accuracy: 0.9236 - val_loss: 0.3163 - val_accuracy: 0.9671
Epoch 2/20
37/36 [==============================] - 21s 559ms/step - loss: 0.1942 - accuracy: 0.9811 - val_loss: 0.1844 - val_accuracy: 0.9802
Epoch 3/20
37/36 [==============================] - 20s 536ms/step - loss: 0.1311 - accuracy: 0.9870 - val_loss: 0.1405 - val_accuracy: 0.9868
Epoch 4/20
37/36 [==============================] - 20s 532ms/step - loss: 0.0986 - accuracy: 0.9911 - val_loss: 0.1404 - val_accuracy: 0.9802
Epoch 5/20
37/36 [==============================] - 20s 531ms/step - loss: 0.0810 - accuracy: 0.9920 - val_loss: 0.1114 - val_accuracy: 0.9894
Epoch 6/20
37/36 [==============================] - 20s 531ms/step - loss: 0.0661 - accuracy: 0.9931 - val_loss: 0.1055 - val_accuracy: 0.9919
Epoch 7/20
37/36 [==============================] - 20s 533ms/step - loss: 0.0538 - accuracy: 0.9952 - val_loss: 0.1798 - val_accuracy: 0.9802

## Unfreeze Model for Fine Tuning

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss = keras.losses.BinaryCrossentropy(from_logits=True) , metrics = [keras.metrics.BinaryAccuracy()])

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=20)

## Evaluate the Model

In [10]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

11/10 [================================] - 4s 352ms/step - loss: 0.0954 - accuracy: 0.9909


[0.09536366164684296, 0.9908814430236816]